In [1]:
!pip install tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 40.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.5/146.5 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 28.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.0/223.0 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.9 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21486 sha256=dcee4daffb9183e0fb891a769af03a8a353229fe45c6872eb93573072fb7abfd
  Stored in directory: /root/.cache/pip/wheels/e1/e8/83/ddea66100678d139b14bc87692ece57c6a2a937956d2532608
Successfully built promise
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.4
    Uninstalling protobuf-3.19.4:
      Successfully uninstalled protobuf-3.19.4
  Attempting uninsta

In [2]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_datasets as tfds

# To Avoid GPU errors
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist', 
    split = ['train', 'test'],
    shuffle_files = True, 
    as_supervised = True, 
    with_info = True
)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
def normalize_img(image, label):
    #normalize images
    return tf.cast(image, tf.float32)/255.0, label

In [5]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)

In [6]:
BATCH_SIZE = 64
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

In [7]:
ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(128)
ds_train = ds_train.prefetch(AUTOTUNE)

In [8]:
model = keras.Sequential([
    keras.Input((28, 28, 1)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.Flatten(),
    layers.Dense(10)
])

In [9]:
save_callback = keras.callbacks.ModelCheckpoint(
    'checkpoint/',
    save_weights_only=True, 
    monitor='accuracy',
    save_best_only=False
)

In [10]:
def scheduler(epoch, lr):
    if epoch < 2:
        return lr
    else:
        return lr * 0.99

In [11]:
lr_scheduler = keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

In [13]:
model.compile(
    optimizer = keras.optimizers.Adam(lr=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [15]:
model.fit(ds_train, epochs=5, verbose=2, callbacks=[save_callback, lr_scheduler])


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/5
938/938 - 11s - loss: 0.2108 - accuracy: 0.9397 - lr: 0.0010 - 11s/epoch - 12ms/step

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/5
938/938 - 5s - loss: 0.0732 - accuracy: 0.9786 - lr: 0.0010 - 5s/epoch - 5ms/step

Epoch 3: LearningRateScheduler setting learning rate to 0.0009900000470224768.
Epoch 3/5
938/938 - 4s - loss: 0.0519 - accuracy: 0.9845 - lr: 9.9000e-04 - 4s/epoch - 4ms/step

Epoch 4: LearningRateScheduler setting learning rate to 0.000980100086890161.
Epoch 4/5
938/938 - 10s - loss: 0.0412 - accuracy: 0.9875 - lr: 9.8010e-04 - 10s/epoch - 11ms/step

Epoch 5: LearningRateScheduler setting learning rate to 0.0009702991275116801.
Epoch 5/5
938/938 - 4s - loss: 0.0315 - accuracy: 0.9905 - lr: 9.7030e-04 - 4s/epoch - 4ms/step


In [16]:
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(logs.keys())

In [17]:
model.compile(
    optimizer = keras.optimizers.Adam(lr=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [19]:
model.fit(ds_train, epochs=5, verbose=2, callbacks=[save_callback, lr_scheduler, CustomCallback()])


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/5
dict_keys(['loss', 'accuracy', 'lr'])
938/938 - 7s - loss: 0.0274 - accuracy: 0.9916 - lr: 0.0010 - 7s/epoch - 7ms/step

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/5
dict_keys(['loss', 'accuracy', 'lr'])
938/938 - 4s - loss: 0.0216 - accuracy: 0.9933 - lr: 0.0010 - 4s/epoch - 4ms/step

Epoch 3: LearningRateScheduler setting learning rate to 0.0009900000470224768.
Epoch 3/5
dict_keys(['loss', 'accuracy', 'lr'])
938/938 - 12s - loss: 0.0157 - accuracy: 0.9956 - lr: 9.9000e-04 - 12s/epoch - 13ms/step

Epoch 4: LearningRateScheduler setting learning rate to 0.000980100086890161.
Epoch 4/5
dict_keys(['loss', 'accuracy', 'lr'])
938/938 - 4s - loss: 0.0121 - accuracy: 0.9968 - lr: 9.8010e-04 - 4s/epoch - 4ms/step

Epoch 5: LearningRateScheduler setting learning rate to 0.0009702991275116801.
Epoch 5/5
dict_keys(['loss', 'accuracy', 'lr'])
938/938 - 4s - loss: 0

In [20]:
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') > 0.90:
            print('Accuracy over 90%, quitting training')
            self.model.stop_training = True
        

In [21]:
model.compile(
    optimizer = keras.optimizers.Adam(lr=0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [22]:
model.fit(ds_train, epochs=5, verbose=2, callbacks=[save_callback, lr_scheduler, CustomCallback()])


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/5
Accuracy over 90%, quitting training
938/938 - 4s - loss: 0.0096 - accuracy: 0.9973 - lr: 0.0010 - 4s/epoch - 4ms/step
